###COMPLETE ANALYSIS FILE BEFORE OUTLIER REMOVAL
1. Upload zipped file with all CSVs
2. Remove cols--"success","trial_type","trial_index","time_elapsed","internal_node_id","recorded_at","source_code_version","ip","user_agent","device","browser","browser_version","platform","platform_version","referer","accept_language"
3. Add the json form info of script self ratings and age of learning
4. Remove participants whose practice accuracy< 80
5. Remove the participants whose self-rated proficiency in reading either script was below 4 on a rating scale of 1-10 (Kaushanskaya et al., 2018)
6. Remove rows with null accuracy in main trials.
7. Remove practice trials
8. Remove main trials with RTs greater than 3000ms and less than 200 ms by uploading the csv file generated by the first block of code



In [ ]:
from google.colab import files
import pandas as pd
import numpy as np
import json, os, zipfile, io, re
from pathlib import Path

In [ ]:
#Block 1
from google.colab import files
import pandas as pd
import numpy as np
import json, os, zipfile, io, re
from pathlib import Path

OUTPUT_CSV_PATH = "/content/combined_clean.csv"
WORK_DIR = "/content/data_zip_extract"
PREFERRED_ROW_INDEX = 2  # 3rd row with response column
DROP_COLUMNS = ["success","trial_type","trial_index","time_elapsed","internal_node_id",
    "recorded_at","source_code_version","ip","user_agent","device",
    "browser","browser_version","platform","platform_version","referer","accept_language"]
FORM_FIELDS = ["dev_start_age","pa_start_age","dev_rating","pa_rating", "form","run_id", "redirected"]

# Upload ZIP
print("Upload ZIP with CSVs:")
uploaded = files.upload()
zip_name = list(uploaded.keys())[0]
zip_path = f"/content/{zip_name}"


with zipfile.ZipFile(zip_path, "r") as zf:
    zf.extractall(WORK_DIR)

# Find CSV
csv_paths = []
for root, _, fs in os.walk(WORK_DIR):
    for f in fs:
        if f.lower().endswith(".csv"):
            csv_paths.append(os.path.join(root, f))
csv_paths.sort()
print(f"Found {len(csv_paths)} CSV files.")

frames, removed = [], []

# Process each CSV
for path in csv_paths:
    # read as strings
    try:
        df = pd.read_csv(path, dtype=str)
    except Exception:
        with open(path, "rb") as f:
            s = f.read().decode("utf-8", errors="replace")
        df = pd.read_csv(io.StringIO(s), dtype=str)

    pid = Path(path).stem

    # Remove entire file if practice accuracy < .80 ----
    if ("trial_type" in df.columns) and ("accuracy" in df.columns):
        prac = df[df["trial_type"].astype(str).str.strip().str.lower() == "practice"].copy()
        if not prac.empty:
            acc_num = pd.to_numeric(prac["accuracy"], errors="coerce")
            acc_mean = acc_num.mean(skipna=True)
            if pd.notna(acc_mean) and acc_mean < 0.80:
                removed.append(Path(path).name)
                continue

    # Extract form fields
    fields = {k: None for k in FORM_FIELDS}
    resp_col = None
    for c in df.columns:
        if "response" in c.lower():
            resp_col = c
            break

    dbg = ""
    if resp_col:
        # try row 3 first
        parsed = None
        if len(df) > PREFERRED_ROW_INDEX:
            cell = str(df.iloc[PREFERRED_ROW_INDEX][resp_col])
            if "{" in cell and "}" in cell:
                try:
                    cell_json = cell[cell.index("{"): cell.rindex("}")+1]
                    try:
                        parsed = json.loads(cell_json)
                    except Exception:
                        # light cleanup: fix curved quotes and trailing commas
                        cell_json2 = re.sub(r"[‘’']", '"', cell_json)
                        cell_json2 = re.sub(r",\s*([}\]])", r"\1", cell_json2)
                        parsed = json.loads(cell_json2)
                except Exception:
                    parsed = None
        if isinstance(parsed, dict) and parsed:
            for k in FORM_FIELDS:
                if parsed.get(k) is not None:
                    fields[k] = parsed[k]
            dbg = f"Used 3rd row of '{resp_col}'."

        # if not found, scan full column: first row containing any of FORM_FIELDS; fallback to first parsable JSON
        if not any(fields.values()):
            best, best_i = None, None
            fallback, fallback_i = None, None
            for i, val in enumerate(df[resp_col].tolist()):
                if not isinstance(val, str) or "{" not in val or "}" not in val:
                    continue
                try:
                    s = val[val.index("{"): val.rindex("}")+1]
                    try:
                        p = json.loads(s)
                    except Exception:
                        s2 = re.sub(r"[‘’']", '"', s)
                        s2 = re.sub(r",\s*([}{])", r"\1", s2)
                        p = json.loads(s2)
                except Exception:
                    p = None
                if isinstance(p, dict) and p:
                    if fallback is None:
                        fallback, fallback_i = p, i
                    if any(k in p for k in FORM_FIELDS):
                        best, best_i = p, i
                        break
            use = best if best is not None else fallback
            used_i = best_i if best is not None else fallback_i
            if isinstance(use, dict):
                for k in FORM_FIELDS:
                    if use.get(k) is not None:
                        fields[k] = use[k]
                dbg = f"Used row {used_i} of '{resp_col}'."
            else:
                dbg = "No parsable JSON found."
    else:
        dbg = "No 'response' column found."

    # ---- Drop columns, add participant_id, and BROADCAST fields ----
    keep = df.drop(columns=[c for c in DROP_COLUMNS if c in df.columns], errors="ignore").copy()
    keep.insert(0, "participant_id", pid)
    for k in FORM_FIELDS:
        keep[k] = fields.get(k)
    frames.append(keep)

# --- Combine ---
combined = pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()

# keep only task_type == 'main'
if "task_type" in combined.columns:
    combined = combined[combined["task_type"].astype(str).str.strip().str.lower() == "main"]

# drop rows with null/empty/"null"/"nan" RT
if "rt" in combined.columns:
    rt_norm = combined["rt"].astype(str).str.strip().str.lower()
    combined = combined[(rt_norm != "") & (rt_norm != "null") & (rt_norm != "nan")]

# remove rows where accuracy == 0
if "accuracy" in combined.columns:
    acc_numeric = pd.to_numeric(combined["accuracy"], errors="coerce")
    combined = combined[acc_numeric != 0]

#remove people who have dev_rating or pa_rating <4
if "dev_rating" in combined.columns and "pa_rating" in combined.columns:
    # Convert columns to numeric, coercing errors will turn non-numeric into NaN
    combined["dev_rating"] = pd.to_numeric(combined["dev_rating"], errors="coerce")
    combined["pa_rating"] = pd.to_numeric(combined["pa_rating"], errors="coerce")
    # Filter out rows where either rating is less than 4 (NaN values will evaluate to False in comparison)
    combined = combined[(combined["dev_rating"] >= 4) & (combined["pa_rating"] >= 4)]

# remove form and run_id
for col in ["form", "run_id", "redirected"]:
    if col in combined.columns:
        combined.pop(col)
# Save & download ---
combined.to_csv(OUTPUT_CSV_PATH, index=False)
print(f"Combined CSV saved: {OUTPUT_CSV_PATH}")

if removed:
    print("Removed files (practice accuracy < 80%):")
    for r in removed:
        print(" -", r)
else:
    print("No files removed for practice accuracy.")

files.download(OUTPUT_CSV_PATH)

Upload ZIP with CSVs:


Saving raw_35.zip to raw_35 (3).zip
Found 35 CSV files.
Combined CSV saved: /content/combined_clean.csv
No files removed for practice accuracy.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

###OUTLIER REMOVED


Type 2: RT< 200ms and RT> 3000ms acc lit

In [ ]:
#Block 2
print("Please upload your CSV file:")
uploaded = files.upload()
filename = list(uploaded.keys())[0]
df = pd.read_csv(filename)

##removing based on rt
before_count = len(df)
df_clean = df[(df['rt'] >= 200) & (df['rt'] <= 3000)]
after_count = len(df_clean)
removed_count = before_count - after_count

output_filename = 'Cleaned_RT_200_3000_removed_35_participants.csv'
df_clean.to_csv(output_filename, index=False)
print(f"Cleaned file saved as: {output_filename}")
print(f"Rows removed: {removed_count}")
files.download(output_filename)


Please upload your CSV file:


Saving combined_chutiyaless_no error.csv to combined_chutiyaless_no error.csv
Cleaned file saved as: Cleaned_RT_200_3000_removed_35_participants.csv
Rows removed: 500


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Model